In [1]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import pickle as pkl
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
nthu_train_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_X", 'rb'))
nthu_train_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_y", 'rb'))

nthu_test_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_X", 'rb'))
nthu_test_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_y", 'rb'))

In [3]:
nthu_train_y = nthu_train_y.mean(axis=1)
nthu_test_y = nthu_test_y.mean(axis=1)

In [4]:
nthu_train_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_train_y, dtype = tf.float32))
nthu_train_y_scaled = nthu_train_y_scaled.numpy()

In [5]:
nthu_test_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_test_y, dtype = tf.float32))
nthu_test_y_scaled = nthu_test_y_scaled.numpy()

In [6]:
scaler = MinMaxScaler()
nthu_train_y_normalized = scaler.fit_transform(nthu_train_y.reshape(-1, 1))

In [7]:
nthu_test_y_normalized = scaler.transform(nthu_test_y.reshape(-1, 1))

In [8]:
idx =[]
np.random.seed(0)
for i in range(10000):
    idx.append(np.random.randint(420953))

In [9]:
n_steps_in, n_features, n_steps_out= 150, 8, 1

# define model
model = Sequential()
model.add(LSTM(100, activation='tanh', input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out, activation='sigmoid'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(nthu_train_X, np.ravel(nthu_train_y_scaled), batch_size=8, epochs=70, verbose=2)

Epoch 1/70
52620/52620 - 360s - loss: 0.0163
Epoch 2/70
52620/52620 - 361s - loss: 0.0075
Epoch 3/70
52620/52620 - 365s - loss: 0.0045
Epoch 4/70
52620/52620 - 370s - loss: 0.0028
Epoch 5/70
52620/52620 - 369s - loss: 0.0020
Epoch 6/70
52620/52620 - 368s - loss: 0.0016
Epoch 7/70
52620/52620 - 366s - loss: 0.0013
Epoch 8/70
52620/52620 - 361s - loss: 0.0011
Epoch 9/70
52620/52620 - 358s - loss: 0.0013
Epoch 10/70
52620/52620 - 358s - loss: 0.0012
Epoch 11/70
52620/52620 - 363s - loss: 0.0078
Epoch 12/70
52620/52620 - 374s - loss: 0.0114
Epoch 13/70
52620/52620 - 361s - loss: 0.0196
Epoch 14/70
52620/52620 - 359s - loss: 0.0235
Epoch 15/70
52620/52620 - 359s - loss: 0.0257
Epoch 16/70
52620/52620 - 359s - loss: 0.0260
Epoch 17/70
52620/52620 - 359s - loss: 0.0256
Epoch 18/70
52620/52620 - 359s - loss: 0.0260
Epoch 19/70
52620/52620 - 359s - loss: 0.0265
Epoch 20/70
52620/52620 - 359s - loss: 0.0266
Epoch 21/70
52620/52620 - 359s - loss: 0.0264
Epoch 22/70
52620/52620 - 358s - loss: 0.02

In [10]:
# demonstrate prediction
yhat = model.predict(nthu_test_X, verbose=1)
print(yhat)

4101/4101 [==============================] - 15s 4ms/step
[[0.76565015]
 [0.7105164 ]
 [0.7115036 ]
 ...
 [0.7526588 ]
 [0.8018341 ]
 [0.7753376 ]]


In [11]:
nthu_test_y

array([0., 0., 0., ..., 2., 2., 2.])

In [12]:
yhat.min()

0.40545344

In [13]:
yhat.shape

(131228, 1)

In [14]:
nthu_test_y.shape

(131228,)

In [15]:
np.unique(yhat)

array([0.40545344, 0.4160897 , 0.41682026, ..., 0.9538388 , 0.9604874 ,
       0.9605762 ], dtype=float32)

In [16]:
np.unique(nthu_test_y_normalized)

array([0.        , 0.01666667, 0.03333333, 0.05      , 0.06666667,
       0.08333333, 0.1       , 0.11666667, 0.13333333, 0.15      ,
       0.16666667, 0.18333333, 0.2       , 0.21666667, 0.23333333,
       0.25      , 0.26666667, 0.28333333, 0.3       , 0.31666667,
       0.33333333, 0.35      , 0.36666667, 0.38333333, 0.4       ,
       0.41666667, 0.43333333, 0.45      , 0.46666667, 0.48333333,
       0.5       , 0.51666667, 0.53333333, 0.55      , 0.56666667,
       0.58333333, 0.6       , 0.61666667, 0.63333333, 0.65      ,
       0.66666667, 0.68333333, 0.7       , 0.71666667, 0.73333333,
       0.75      , 0.76666667, 0.78333333, 0.8       , 0.81666667,
       0.83333333, 0.85      , 0.86666667, 0.88333333, 0.9       ,
       0.91666667, 0.93333333, 0.95      , 0.96666667, 0.98333333,
       1.        ])

In [17]:
tf.keras.activations.sigmoid(tf.constant(1, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.7310586>

In [18]:
def logit(x):
    """ Computes the logit function, i.e. the logistic sigmoid inverse. """
    if (x<=0.5):
        return float(0)
    if (x>=0.880797):
        return float(2)
    return float(- tf.math.log(1. / x - 1.))

In [19]:
yhat_inversed = np.array([logit(x) for x in yhat])

In [20]:
(yhat==1).sum()

0

In [21]:
yhat_inversed2 = scaler.inverse_transform(yhat)

In [22]:
mean_squared_error(yhat_inversed,nthu_test_y)

0.9314243693924137

1.3878689692067376

2.482321472640477

3.3216619367005453

tanh + sigmoid: 1.108981281677717

tanh + minmax: 1.5659759629822405


In [23]:
1.3878689692067376**0.5

1.1780785072340203

In [24]:
model.save("lstm_vanilla.h5")